In [1]:
# 1. Core PyTorch first
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# 2. Utility libs
!pip install matplotlib timm

# 3. Segment Anything
!pip install git+https://github.com/facebookresearch/segment-anything.git

# 4. CLIP
!pip install ftfy regex

# 5. OpenCV (with contrib modules for tracker)
!pip install opencv-contrib-python



Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/facebookresearch/segment-anything.git to c:\users\hello\appdata\local\temp\pip-req-build-01kndioo
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git 'C:\Users\Hello\AppData\Local\Temp\pip-req-build-01kndioo'


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import numpy as np
import cv2
import torch
import timm
from segment_anything import sam_model_registry, SamPredictor
import timm
import torch.nn as nn
import torchvision.transforms as T
from PIL import Image
import clip

In [2]:
!curl.exe -L "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth" -o "sam_vit_b.pth"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  357M    0 1194k    0     0   841k      0  0:07:15  0:00:01  0:07:14  846k
  0  357M    0 2858k    0     0  1179k      0  0:05:10  0:00:02  0:05:08 1183k
  1  357M    1 4682k    0     0  1365k      0  0:04:28  0:00:03  0:04:25 1368k
  1  357M    1 6650k    0     0  1506k      0  0:04:03  0:00:04  0:03:59 1508k
  2  357M    2 8698k    0     0  1601k      0  0:03:48  0:00:05  0:03:43 1795k
  2  357M    2 10.4M    0     0  1662k      0  0:03:40  0:00:06  0:03:34 1895k
  3  357M    3 12.1M    0     0  1682k      0  0:03:37  0:00:07  0:03:30 1925k
  3  357M    3 13.7M    0     0  1674k      0  0:03:38  0:00:08  0:03:30 1886k
  4  357M    4 16.1M    0     0  1757k      0  0:03

In [3]:
import os
print(os.path.exists("sam_vit_b.pth"))  # Should be True


True


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# load model (point to the checkpoint you downloaded)
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b.pth")
sam.to(device)
predictor = SamPredictor(sam)

In [5]:
# --------------------------
# 2. CLIP (Option 3: re-implement + load weights)
# --------------------------
!pip install git+https://github.com/openai/CLIP.git
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

# import open_clip
# clip_model, _, clip_preprocess = open_clip.create_model_and_transforms(
#     'ViT-B-32', pretrained='openai'
# )
# clip_model = clip_model.eval().cuda()

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to c:\users\hello\appdata\local\temp\pip-req-build-o0u51i1g
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Hello\AppData\Local\Temp\pip-req-build-o0u51i1g'


In [6]:
clip_model = clip_model.eval().to(device)

In [7]:
# --------------------------
# 3. DINO (Option 3: re-implement + load weights)
# --------------------------
import timm
dino_model = timm.create_model('vit_base_patch16_224', pretrained=True)
dino_model.eval().to(device)

# Preprocess for DINO
dino_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


In [8]:
# --------------------------
# Helper functions
# --------------------------

def get_sam_mask(image, point):
    # point = (x, y)
    predictor.set_image(image)

    input_point = np.array([[point[0], point[1]]])
    input_label = np.array([1])   # 1 = foreground

    masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True
    )
    
    # just return the best scoring mask
    best_mask = masks[np.argmax(scores)]
    return best_mask.astype(np.uint8)


def get_clip_score(segment_crop, text_query):
    img_proc = clip_preprocess(segment_crop).unsqueeze(0).cuda()
    text = open_clip.tokenize([text_query]).cuda()
    with torch.no_grad():
        img_feat = clip_model.encode_image(img_proc)
        txt_feat = clip_model.encode_text(text)
        img_feat /= img_feat.norm(dim=-1, keepdim=True)
        txt_feat /= txt_feat.norm(dim=-1, keepdim=True)
        score = (img_feat @ txt_feat.T).item()
    return score

def get_dino_score(segment_crop, ref_image):
    img1 = dino_transform(segment_crop).unsqueeze(0).cuda()
    img2 = dino_transform(ref_image).unsqueeze(0).cuda()
    with torch.no_grad():
        f1 = dino_model(img1)
        f2 = dino_model(img2)
        f1 /= f1.norm(dim=-1, keepdim=True)
        f2 /= f2.norm(dim=-1, keepdim=True)
        score = (f1 @ f2.T).item()
    return score

In [9]:
!pip install -U ipywidgets
import ipywidgets as widgets


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
#interactive part
# --------------------------
# Load video frame
# --------------------------
cap = cv2.VideoCapture("C:\\Users\\Hello\\Downloads\\example_videos_car_following.avi") 
ret, frame = cap.read()
cap.release()

if not ret:
    raise RuntimeError("Failed to read video frame.")

image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# --------------------------
# Mouse click handling
# --------------------------
click_point = None

def mouse_callback(event, x, y, flags, param):
    global click_point
    if event == cv2.EVENT_LBUTTONDOWN:
        click_point = (x, y)
        print(f"Clicked at: {click_point}")
        cv2.destroyAllWindows()  # close window immediately after click

# --------------------------
# Segmentation runner
# --------------------------
def run_segmentation(choice):
    global click_point

    if choice.lower() == 'click':
        print("Click on the object in the window...")
        cv2.imshow("Click on an object", cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))
        cv2.setMouseCallback("Click on an object", mouse_callback)
        cv2.waitKey(0)  # wait for click

        if click_point is None:
            print("No click detected.")
            return
        
        # Segment object with SAM
        mask = get_sam_mask(image_rgb, click_point)

        # Overlay mask on original frame
        overlay = image_rgb.copy()
        overlay[mask > 0] = [255, 0, 0]  # red overlay where mask=1
        segmented = cv2.addWeighted(image_rgb, 0.7, overlay, 0.3, 0)

        # Show the segmented image
        Image.fromarray(segmented).show()
        print("Segmented object highlighted in frame.")


    elif choice.lower() == 'text':
        text_query = input("Enter text query: ")
        h, w, _ = image_rgb.shape

        # Dummy mask from center
        mask = get_sam_mask(image_rgb, (w//2, h//2))
        x, y, w, h = cv2.boundingRect(mask.astype(np.uint8))
        crop = Image.fromarray(image_rgb[y:y+h, x:x+w])
        crop.show()

        score = get_clip_score(crop, text_query)
        print(f"CLIP score for '{text_query}': {score:.4f}")

    elif choice.lower() == 'reference':
        ref_path = input("Enter path to reference image: ")
        ref_image = Image.open(ref_path).convert("RGB")

        h, w, _ = image_rgb.shape
        mask = get_sam_mask(image_rgb, (w//2, h//2))
        x, y, w, h = cv2.boundingRect(mask.astype(np.uint8))
        crop = Image.fromarray(image_rgb[y:y+h, x:x+w])
        crop.show()

        score = get_dino_score(crop, ref_image)
        print(f"DINO score vs reference: {score:.4f}")

    else:
        print("Invalid mode. Choose Click, Text, or Reference.")

# --------------------------
# Main
# --------------------------
mode = input("Choose mode (Click / Text / Reference): ")
run_segmentation(mode)


Click on the object in the window...
Clicked at: (128, 98)
